In [1]:
import numpy as np
import os , sys
sys.path.append("..")

In [2]:
from Gen_src.diffusion import GaussianDiffusion1D
from Gen_src.model import Unet1D
import torch
import os,sys
# Add the parent directory of Gen_src to sys.path
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../.."))
import torch
import os

# Same architecture as in training
model = Unet1D(
    dim=32,
    dim_mults=(2, 4, 8, 16),
    channels=2,  # X and Y
    dropout=0.1
).cuda()  # or .to(device)

# Create the same diffusion wrapper
diffusion = GaussianDiffusion1D(
    model,
    seq_length=192,      # must match your training setup
    objective='pred_noise',
    timesteps=1000
).cuda()  # or .to(device)

# Load checkpoint
checkpoint_path = rf"/home/bardiya/projects/diffusion_air_manifolding/codes/creative-generativeai-diffusion/model_weigths/model_epoch_375.pt"
model.load_state_dict(torch.load(checkpoint_path, weights_only=True))
model.eval()
print("Loaded model weights from:", checkpoint_path)


/home/bardiya/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/bardiya/projects/diffusion_air_manifolding/codes/creative-generativeai-diffusion/src/optimization_loop/../Gen_src/diffusion.py:331: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)


Loaded model weights from: /home/bardiya/projects/diffusion_air_manifolding/codes/creative-generativeai-diffusion/model_weigths/model_epoch_375.pt


In [3]:
from pathlib import Path

DATA_DIR = Path(rf"/home/bardiya/projects/diffusion_air_manifolding/codes/creative-generativeai-diffusion/data/raw")
coord_mm = np.load(DATA_DIR/"coord_min_max.npy")  # [[x_min,y_min],[x_max,y_max]]
x_min,y_min = coord_mm[0]; x_max,y_max = coord_mm[1]

def inv_coords(xs_s):                   # xs_s shape (...,2,192) tensor
    xs_np = xs_s.permute(0,2,1).cpu().numpy()    # -> (B,192,2)
    xs_np[...,0] = xs_np[...,0]*(x_max-x_min) + x_min
    xs_np[...,1] = xs_np[...,1]*(y_max-y_min) + y_min
    return xs_np                                # (B,192,2) numpy


In [4]:
num_to_generate = 2
batch_size      = 2

# generated_scaled = []
# latent = torch.randn((1,2,192)).to("cuda")
# samples1 = diffusion.latent_sample(latent , is_ddim=True)
# samples2 = diffusion.latent_sample(latent , is_ddim=True)
# generated_samples1 = inv_coords(samples1)
# generated_samples2 = inv_coords(samples2)


all_latent = []
all_shapes = []
with torch.no_grad():
    done = 0
    while done < num_to_generate:
        cur = min(batch_size, num_to_generate - done)
        
        latent = torch.randn((cur,2,192)).to("cuda")
        samples = diffusion.latent_sample(latent , is_ddim=True)
        generated_real = inv_coords(samples)
        
        all_latent.append(latent.cpu().detach().numpy())
        all_shapes.append(generated_real)
        done += cur
        print(f"Generated {done}/{num_to_generate}")

sampling loop time step: 100%|██████████| 1000/1000 [00:13<00:00, 74.05it/s]

Generated 2/2


In [5]:
docker_mount_path = "/home/bardiya/projects/diffusion_air_manifolding/codes/Airfoil_MPI_system"
# docker_mount_path = "."

np.save(os.path.join(docker_mount_path , "DB2.npy") , {
    "latents": np.vstack(all_latent),
    "shapes": np.vstack(all_shapes),
    "performances": None
})

In [11]:
sys.path.append("../OpenFoam")
from OpenFoam.Airfoil_simulation_1.ShapeToPerformance import shape_to_performance as STP1
docker_container_id = "78257c051b5b" 
# command = fr"docker exec {docker_container_id} python3 /home/airfoil_UANA/performance_finding.py"
command = (
    f'docker exec {docker_container_id} bash -c "'
    f'source /opt/openfoam5/etc/bashrc && '
    f'cd /home/airfoil_UANA && '
    f'python3 performance_finding.py"'
)

os.system(command)


(2, 192, 2)
8
core 1:
sample 1:
	Using len 40.000 angle +0.000 
	Resulting freestream vel x,y: 40.0,-0.0
error during mesh creation!
	mesh generation failed, aborting
-1000
1000
/home/airfoil_UANA
/home/airfoil_UANA
core 0:
sample 0:
	Using len 40.000 angle +0.000 
	Resulting freestream vel x,y: 40.0,-0.0
error during mesh creation!
	mesh generation failed, aborting
-1000
1000
/home/airfoil_UANA
/home/airfoil_UANA
0.txt
1.txt
[0 1]
[[-1000.  1000.     0.]
 [-1000.  1000.     1.]]
done!!!


Error   : Could not find extruded vertex (0.9762810542449498, -0.007106960108275135, 1) in surface 1026
Error   : Could not find extruded vertex (0.9846254750024874, -0.006615931396918012, 1) in surface 1026
Error   : Could not find extruded vertex (0.9762810542449498, -0.007106960108275135, 1) in surface 1026
Error   : Could not find extruded vertex (0.9846254750024874, -0.006615931396918012, 1) in surface 1026
Error   : Could not find extruded vertex (0.9762810542449498, -0.007106960108275135, 1) in surface 1026
Error   : Could not find extruded vertex (0.9846254750024874, -0.006615931396918012, 1) in surface 1026
Error   : Could not find extruded vertex (0.9762810542449498, -0.007106960108275135, 1) in surface 1026
Error   : Could not find extruded vertex (0.9846254750024874, -0.006615931396918012, 1) in surface 1026
Error   : Could not find extruded vertex (0.9762810542449498, -0.007106960108275135, 1) in surface 1026
Error   : Could not find extruded vertex (0.9846254750024874, -0

0

In [13]:
performance_path =os.path.join(docker_mount_path , rf"performance.npy")
perfromance = np.load(performance_path,allow_pickle=True)
perfromance

array([[-1000.,  1000.,     0.],
       [-1000.,  1000.,     1.]])